In [59]:
import pandas as pd
import numpy as np
from sklearn import svm
from sklearn.model_selection import GridSearchCV, KFold, cross_validate
from sklearn import linear_model
from sklearn import preprocessing
from sklearn import cross_decomposition
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import smogn

### Constants

In [40]:
data_url =  "https://raw.githubusercontent.com/Naio/aasa-stability-prediction/master/data/processed/"
#Seed for controlling any random procedure during the experiments
seed = 99

### Defining hyperparameter grids
Each algorithm has its corresponding hyperparameter grid for later use in grid search inner cross-validation loop.

In [42]:
least_squares_grid = {} #Ordinary least square doesn't have hyperparamters

ridge_grid = {'alpha' : np.logspace(-6, 6, 13)} #Alpha between 1.e-06 and 1.e+06
lasso_grid = {'alpha' : np.logspace(-6, 6, 13)} 
pls_grid = {'n_components': np.linspace(start = 2, stop=25, num=24).astype(int)} #Between 2 and 25 Principal Components

svr_grid = [
    #Grid for linear kernel
    #{'C': np.logspace(-6, 6, 13), 'kernel': ['linear'],'epsilon' :np.linspace(start = 1.0, stop=3.2, num=12)},
    #Grid for rbf and sigmoid kernel
    {'C': np.logspace(-6, 6, 13), 'gamma': np.logspace(-6, 6, 13), 'kernel': ['rbf'], 
     'epsilon' :np.linspace(start = 1.0, stop=2.5, num=16)},
    #Grid for polinomial kernel
    {'C': np.logspace(-6, 6, 13), 'gamma': np.logspace(-6, 6, 13), 'kernel': ['poly'], 'degree': [2,3], 
     'epsilon' :np.linspace(start = 1.0, stop=2.5, num=16)}
 ]


### Creating estimators for each learning method

In [43]:
#learning_methods function will create the empty estimators and map them to their corresponding hyperparameter grid.
def get_learning_methods():
    learning_methods = [{'name': 'OLS', 'estimator': linear_model.LinearRegression(), 'hyperparameter_grid': least_squares_grid},
                        {'name': 'RIDGE','estimator':linear_model.Ridge(random_state=seed), 'hyperparameter_grid': ridge_grid},
                        {'name': 'LASSO', 'estimator':linear_model.Lasso(), 'hyperparameter_grid': lasso_grid},
                        {'name': 'PLS', 'estimator':cross_decomposition.PLSRegression(), 'hyperparameter_grid': pls_grid},
                        {'name': 'SVR', 'estimator':svm.SVR(), 'hyperparameter_grid': svr_grid}]
    return learning_methods

### Loading the data

In [44]:
def datasets():
    
    datasets_names = ['A', 'B', 'C', 'D']
    
    for dataset_name in datasets_names:
        protein_dataset = pd.read_csv(data_url + dataset_name + '.csv')
        proteins_X = protein_dataset.iloc[:, 1:].to_numpy()
        proteins_X = preprocessing.scale(proteins_X) #Z-Score standarization
        proteins_y = protein_dataset.iloc[:,0].to_numpy()
        
        yield {'name': dataset_name, 'features': proteins_X, 'target': proteins_y}

In [ ]:
def load_dataset(name):
    return 

### Nested CV

In [57]:
#The nested_cv function returns a dictionary like with the outer cross-validation loop scores for every learning method.
#The dictionary looks like:
#{
#    'r2': {'PLS': [0.99,...,0.67], 'SVR': [0.94,..., 0.98], ... , 'OLS': [0.4, ..., 0.32]}, 
#  'rmse': {'PLS': [1.297116,...,2.297116], 'SVR': [1.291,..., 0.29471], ... , 'OLS': [3.19283, ..., 5.827391]}
#}
def nested_cv(features, target):
    
    #Score metric used for hyperparameter optimization in inner CV loop
    inner_scoring = 'r2'
    
    learning_methods = get_learning_methods()
    
    results = {}
    for learning_method in learning_methods:
        
        inner_cv = KFold(n_splits=10, shuffle=True, random_state=seed)
        outer_cv = KFold(n_splits=10, shuffle=True, random_state=seed)
        
        #Contains data about the results for a particular learning method.
        learning_method_results = {}
        
        learning_method_results['best_parameters'] = []
        learning_method_results['train_scores'] = {'R-Squared': [], 'RMSE': []}
        learning_method_results['test_scores'] = {'R-Squared': [], 'RMSE': []}
        
        for train_index, test_index in outer_cv.split(features):
            
            #Split the data between train and test sets
            train_features, test_features = features[train_index], features[test_index]
            train_target, test_target = target[train_index], target[test_index]
            
            
            #When the fit() method is called, it will internally perform a grid search cross-validation. 
            #Once it finds the best hyperparameters, it will fit on complete training set using those parameters.
            grid_search_estimator = GridSearchCV(estimator = learning_method['estimator'], 
                           param_grid = learning_method['hyperparameter_grid'], 
                           cv = inner_cv, 
                           scoring = inner_scoring,
                           #When n_jobs is -1, all CPUs are used to run cross-validation in parallel
                           n_jobs=-1)
            
            grid_search_estimator.fit(train_features, train_target)
            best_parameters = grid_search_estimator.best_params_
            learning_method_results['best_parameters'].append(best_parameters)
            
            #Prediction using the best estimator selected via Grid Search CV
            train_prediction = grid_search_estimator.predict(train_features)
            test_prediction = grid_search_estimator.predict(test_features)
            
            
            #Calculating R-Squared score
            train_r2 = r2_score(y_true = train_target, y_pred = train_prediction)
            test_r2 = r2_score(y_true = test_target, y_pred = test_prediction)
            
            learning_method_results['train_scores']['R-Squared'].append(train_r2)
            learning_method_results['test_scores']['R-Squared'].append(test_r2)
            
            #Calculating RMSE score
            train_rmse =  mean_squared_error(y_true = train_target, y_pred = train_prediction, squared=False)
            test_rmse = mean_squared_error(y_true = test_target, y_pred = test_prediction, squared=False)
            
            learning_method_results['train_scores']['RMSE'].append(train_rmse)
            learning_method_results['test_scores']['RMSE'].append(test_rmse)
            
        
            
        #Scores for particular learning method
        results[learning_method['name']] = learning_method_results
    return results

In [53]:
def display_results(results):
    learning_methods = get_learning_methods()
    learning_methods_names = [learning_method['name'] for learning_method in learning_methods]
    
    
    train_r2 = {method_name:results[method_name]['train_scores']['R-Squared'] for method_name in learning_methods_names} 
    train_rmse = {method_name:results[method_name]['train_scores']['RMSE'] for method_name in learning_methods_names}
    
    test_r2 = {method_name:results[method_name]['test_scores']['R-Squared'] for method_name in learning_methods_names} 
    test_rmse = {method_name:results[method_name]['test_scores']['RMSE'] for method_name in learning_methods_names}
    
    best_parameters = {method_name:results[method_name]['best_parameters'] for method_name in learning_methods_names}
    
    test_scores = {}

    print("---------- Best parameters--------------")
    display(pd.DataFrame(best_parameters))
    print("------------- Train R-Squared --------------")
    display(pd.DataFrame(train_r2))
    print("------------- Train RMSE -------------------")
    display(pd.DataFrame(train_rmse))
    print("------------- Test R-Squared ---------------")
    test_scores['test_r2'] = pd.DataFrame(test_r2)
    display(pd.DataFrame(test_r2))
    print("------------- Test RMSE --------------------")
    display(pd.DataFrame(test_rmse))
    test_scores['test_rmse'] = pd.DataFrame(test_rmse)
    

    return test_scores
    
    

### Non-nested CV

In [54]:
def non_nested_cv(features, target, estimator):
    
    cv = KFold(n_splits=10, shuffle=True, random_state=seed)
    for train_index, test_index in cv.split(features):
        features_train, features_test = features[train_index], features[test_index]
        target_train, target_test = target[train_index], target[test_index]

        estimator.fit(features_train, target_train)

        prediction = estimator.predict(features_test)

        score = r2_score(y_true=target_test,y_pred = prediction)
        print(score)

### Running experiments

In [58]:
general_results = {}
for dataset in datasets():
    print("Dataset ", dataset['name'])
    general_results[dataset['name']] = nested_cv(dataset['features'], dataset['target'])
    #non_nested_cv(dataset['features'], dataset['target'], estimator = svm.SVR(epsilon = 1.0,C= 10, gamma = 0.25, 
    #                                                                         kernel='rbf'))

A
Best parameters


,OLS,RIDGE,LASSO,PLS,SVR
0,{},{'alpha': 1000.0},{'alpha': 1.0},{'n_components': 2},"{'C': 100000.0, 'epsilon': 2.1, 'gamma': 1e-06..."
1,{},{'alpha': 100.0},{'alpha': 1.0},{'n_components': 2},"{'C': 10000.0, 'epsilon': 2.1, 'gamma': 1e-05,..."
2,{},{'alpha': 100.0},{'alpha': 1.0},{'n_components': 2},"{'C': 10000.0, 'epsilon': 1.9, 'gamma': 1e-06,..."
3,{},{'alpha': 100.0},{'alpha': 0.1},{'n_components': 2},"{'C': 10.0, 'epsilon': 1.4, 'gamma': 0.01, 'ke..."
4,{},{'alpha': 1000.0},{'alpha': 1.0},{'n_components': 2},"{'C': 100000.0, 'epsilon': 2.1, 'gamma': 1e-06..."
5,{},{'alpha': 1000.0},{'alpha': 1.0},{'n_components': 2},"{'C': 100000.0, 'epsilon': 2.1, 'gamma': 1e-06..."
6,{},{'alpha': 1000.0},{'alpha': 1.0},{'n_components': 2},"{'C': 100000.0, 'epsilon': 2.1, 'gamma': 1e-06..."
7,{},{'alpha': 1000.0},{'alpha': 1.0},{'n_components': 2},"{'C': 100000.0, 'epsilon': 2.1, 'gamma': 1e-06..."
8,{},{'alpha': 100.0},{'alpha': 0.1},{'n_components': 2},"{'C': 1.0, 'epsilon': 1.0, 'gamma': 0.01, 'ker..."
9,{},{'alpha': 1000.0},{'alpha': 1.0},{'n_components': 2},"{'C': 1.0, 'epsilon': 1.0, 'gamma': 0.01, 'ker..."


Train R-Squared


,OLS,RIDGE,LASSO,PLS,SVR
0,1.0,0.301148,0.162065,0.499519,0.401804
1,1.0,0.497844,0.109458,0.494580,0.370625
2,1.0,0.536888,0.154232,0.523139,0.443027
3,1.0,0.611952,0.666535,0.600326,0.645046
4,1.0,0.285157,0.126260,0.486459,0.371792
5,1.0,0.293325,0.102648,0.483777,0.374668
6,1.0,0.278177,0.102856,0.462785,0.371044
7,1.0,0.244740,0.023091,0.444062,0.334504
8,1.0,0.550458,0.653020,0.530838,0.609218
9,1.0,0.288601,0.106047,0.482294,0.551431


Train RMSE


...

,OLS,RIDGE,LASSO,PLS,SVR
0,2.534884,3.688265,4.225022,3.845184,2.944456
1,2.455398,3.718541,4.221187,3.600216,3.082372
2,2.500078,3.558819,3.939531,3.693299,3.150637
3,2.468320,3.774887,4.268724,3.534782,2.282781
4,2.557221,3.853351,4.332308,3.720678,3.320642
5,2.434188,3.598966,4.114641,3.765252,2.832552
6,2.491442,3.767978,3.561458,3.652120,3.250334
7,2.560710,3.762824,4.237385,3.576155,1.873143
8,2.433592,3.704088,3.540992,3.845624,3.252990
9,2.511967,3.689320,4.184596,3.860767,2.143696


Test R-Squared


,OLS,RIDGE,LASSO,PLS,SVR
0,0.616913,0.412194,0.182205,0.417055,0.430583
1,-0.124858,0.004747,0.170919,-0.169958,0.537353
2,0.562887,0.140100,0.020451,0.085326,0.429966
3,-2.150568,-0.593383,-0.464474,-1.324082,-0.781115
4,-1.490364,0.192425,0.000423,-0.226193,0.115154
5,-0.223474,-0.069120,0.038935,-0.323785,0.151602
6,-0.204103,0.372699,0.431299,0.435648,0.332192
7,-0.261126,-0.074966,-0.171277,-0.380899,-1.249043
8,0.229245,0.318900,0.426834,0.081635,0.449236
9,0.334200,0.336211,0.178179,0.299680,-0.081063


Test RMSE


,OLS,RIDGE,LASSO,PLS,SVR
0,3.597821,4.456640,5.256697,4.438174,4.386376
1,4.612987,4.339105,3.960335,4.704554,2.958409
2,3.848021,5.397153,5.760415,5.566391,4.394313
3,4.456255,3.169098,3.038201,3.827377,3.350592
4,3.529375,2.009823,2.236014,2.476542,2.103780
5,5.498058,5.139551,4.872909,5.719007,4.578379
6,4.790288,3.457545,3.292091,3.279478,3.567432
7,3.471925,3.205446,3.345961,3.633055,4.636497
8,4.810756,4.522312,4.148541,5.251251,4.066659
9,4.247445,4.241023,4.718939,4.356161,5.412292


In [ ]:
test_scores_A = display_results(general_results['A'])

In [ ]:
%matplotlib inline
test_scores_A['test_r2'].iloc[:,1:].boxplot()

In [92]:
protein_dataset = pd.read_csv(data_url + 'B' + '.csv')
test_set = protein_dataset.iloc[[20,32,43,47],:]
train_index = protein_dataset.index.isin([20,32,43,47])
train_set = protein_dataset.loc[~train_index,:]


In [93]:
def min_max_df(df):
    maximum = df.max().max()
    minimum = df.min().min()
    
    def min_max_scaler(value):
        return (value-minimum)/(maximum - minimum)
    
    return df.applymap(min_max_scaler)


In [94]:
scaled_train = min_max_df(train_set)
scaled_test = min_max_df(test_set)

In [95]:
train_X = scaled_train.iloc[:, 1:].to_numpy()
train_y = scaled_train.iloc[:,0].to_numpy()
test_X = scaled_test.iloc[:, 1:].to_numpy()
test_y = scaled_test.iloc[:,0].to_numpy()

In [96]:
svr = svm.SVR(kernel="rbf", epsilon=1, C=10, gamma=0.25)

svr.fit(train_X, train_y)

SVR(C=10, cache_size=200, coef0=0.0, degree=3, epsilon=1, gamma=0.25,
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [97]:
prediction = svr.predict(test_X)
prediction

array([0.00020242, 0.00020242, 0.00020242, 0.00020242])

In [98]:
test_y

array([0.00000000e+00, 1.92543669e-05, 5.29495091e-05, 6.59462068e-05])

In [100]:
score = mean_squared_error(y_true=test_y,y_pred = prediction, squared=False)

In [101]:
score

0.00016992101577140912

### Testing Pipeline object works


In [ ]:
mlr1 = linear_model.LinearRegression()
mlr1.fit(scaled_proteins_X,proteins_y)

mlr2 = linear_model.LinearRegression()
pipeline_estimator = [('standarization', StandardScaler()), ('mlr', mlr2)]
pipe = Pipeline(pipeline_estimator)
pipe.fit(proteins_X, proteins_y)


#mlr1 and mlr2 should be equal
